In [9]:
import zipfile
import os

zip_path = "Drug_dataset.zip"   # change to your filename
extract_path = "drug_data"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Files extracted to:", os.listdir(extract_path))


Files extracted to: ['Medicine_Details.csv']


In [11]:
import pandas as pd

# Replace with your actual CSV name inside the zip
csv_file = os.path.join(extract_path, "/content/Drug_dataset/Medicine_Details.csv")
df = pd.read_csv(csv_file)

print("Columns:", df.columns)
print(df.head())


Columns: Index(['Medicine Name', 'Composition', 'Uses', 'Side_effects', 'Image URL',
       'Manufacturer', 'Excellent Review %', 'Average Review %',
       'Poor Review %'],
      dtype='object')
              Medicine Name  \
0   Avastin 400mg Injection   
1  Augmentin 625 Duo Tablet   
2       Azithral 500 Tablet   
3          Ascoril LS Syrup   
4         Aciloc 150 Tablet   

                                         Composition  \
0                                Bevacizumab (400mg)   
1    Amoxycillin  (500mg) +  Clavulanic Acid (125mg)   
2                               Azithromycin (500mg)   
3  Ambroxol (30mg/5ml) + Levosalbutamol (1mg/5ml)...   
4                                 Ranitidine (150mg)   

                                                Uses  \
0   Cancer of colon and rectum Non-small cell lun...   
1                  Treatment of Bacterial infections   
2                  Treatment of Bacterial infections   
3                      Treatment of Cough with mucus   

In [13]:
df['doc_text'] = df.apply(
    lambda row: f"Drug: {row['Medicine Name']}, Composition: {row['Composition']}, Uses: {row['Uses']}",
    axis=1
)

# Make sure the embedder is initialized before this cell
# doc_embeddings = embedder.encode(df['doc_text'].tolist(), normalize_embeddings=True)

# import faiss
# index = faiss.IndexFlatIP(doc_embeddings.shape[1])
# index.add(doc_embeddings.astype('float32'))

In [14]:
def query_drug(input_text, top_k=3):
    """
    Takes a drug+dosage text, finds the most relevant entries in your dataset,
    and returns their info (usage, therapy, etc).
    """
    # Encode query
    query_emb = embedder.encode([input_text], normalize_embeddings=True)

    # Search FAISS index
    D, I = index.search(query_emb.astype('float32'), top_k)

    results = []
    for idx, score in zip(I[0], D[0]):
        if idx == -1:  # no result
            continue
        row = df.iloc[idx]
        results.append({
            "Drug": row.get("DrugName", ""),
            "Dosage": row.get("Dosage", ""),
            "Indication": row.get("Indication", ""),
            "Therapy": row.get("Therapy", ""),
            "SideEffects": row.get("SideEffects", ""),
            "Score": round(float(score), 3)
        })
    return results


In [17]:
from sentence_transformers import SentenceTransformer

# Initialize an embedding model
embedder = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
!pip install sentence-transformers

In [21]:
!pip uninstall -y websockets
!pip install websockets==15.0.1 gradio --upgrade


Found existing installation: websockets 12.0
Uninstalling websockets-12.0:
  Successfully uninstalled websockets-12.0
  Using cached websockets-15.0.1-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
Using cached websockets-15.0.1-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (182 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.6/324.6 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 1.3.0
    Uninstalling gradio_client-1.3.0:
      Successfully uninstalled gradio_client-1.3.0
  Attempting uninstall: gradio
    Found existing installation: gradio 4.44.0
    Uninstalling gradio-4.44.0:
      Successfully uninstalled gradio-4.44.0


In [24]:
print(df.columns.tolist())


['Medicine Name', 'Composition', 'Uses', 'Side_effects', 'Image URL', 'Manufacturer', 'Excellent Review %', 'Average Review %', 'Poor Review %', 'doc_text']


In [27]:
def compute_avg_review(row):
    try:
        exc = float(row['Excellent Review %']) if 'Excellent Review %' in df.columns else 0
        avg = float(row['Average Review %']) if 'Average Review %' in df.columns else 0
        poor = float(row['Poor Review %']) if 'Poor Review %' in df.columns else 0

        # Weighted score (out of 5)
        score = (exc*5 + avg*3 + poor*1) / (exc + avg + poor + 1e-6)
        return round(score, 2)
    except:
        return "Not available"


In [ ]:
DISCLAIMER = (
    "⚠️ **Disclaimer:** Educational reference only. "
    "Not medical advice. Always verify with official labels and consult a licensed clinician."
)

def search_drug_info_cards(user_input):
    query_emb = embedder.encode([user_input], normalize_embeddings=True)
    D, I = index.search(query_emb.astype('float32'), 3)

    if I[0][0] == -1:
        return "❌ No match found in dataset.\n\n" + DISCLAIMER

    results_md = ""
    for idx, score in zip(I[0], D[0]):
        row = df.iloc[idx]

        drug_name   = row['Medicine Name'] if 'Medicine Name' in df.columns else "Not available"
        composition = row['Composition'] if 'Composition' in df.columns else "Not available"
        manufacturer= row['Manufacturer'] if 'Manufacturer' in df.columns else "Not available"
        indication  = row['Uses'] if 'Uses' in df.columns else "Not available"
        side_effects= row['Side_effects'] if 'Side_effects' in df.columns else "Not available"
        therapy     = "Not available"
        avg_review  = compute_avg_review(row)

        results_md += f"""
<details>
<summary>💊 **{drug_name} ({composition})**</summary>

🏭 **Manufacturer:** {manufacturer}
⭐ **Average Review Score:** {avg_review} / 5
📌 **Usage/Indication:** {indication}
🩺 **Therapy:** {therapy}
⚠️ **Side Effects:** {side_effects}
🔢 **Similarity Score:** {round(float(score),3)}

</details>

---
"""

    # Append disclaimer at the end
    results_md += f"\n\n---\n\n{DISCLAIMER}"
    return results_md


with gr.Blocks() as demo:
    gr.Markdown("# 💊 Drug Information Assistant")
    gr.Markdown("Enter a **drug composition** and get its usages, manufacturer, side effects, and review score in expandable cards.")

    inp = gr.Textbox(label="Enter Medicine Composition")
    out = gr.Markdown()

    btn = gr.Button("Search Drug Info")
    btn.click(fn=search_drug_info_cards, inputs=inp, outputs=out)

    # Show disclaimer always at the bottom of interface too
    gr.Markdown(DISCLAIMER, elem_classes="disclaimer")

demo.launch()
